In [1]:
from fastai import *
from fastai.vision.all import *

In [2]:
plt.rcParams['figure.figsize'] = (20,10)
plt.rcParams['figure.dpi'] = 150

In [3]:
path = Path('data/pic/src/640/base/')

In [5]:
list(path.ls())

[Path('data/pic/src/640/base/base_incorrect_green2_21.JPG'),
 Path('data/pic/src/640/base/base_incorrect_white_37.JPG'),
 Path('data/pic/src/640/base/base_incorrect_black_62.JPG'),
 Path('data/pic/src/640/base/base_correct_stone_16.JPG'),
 Path('data/pic/src/640/base/base_incorrect_white_43.JPG'),
 Path('data/pic/src/640/base/base_correct_black_60.JPG'),
 Path('data/pic/src/640/base/base_correct_blue_67.JPG'),
 Path('data/pic/src/640/base/base_incorrect_wood2_51.JPG'),
 Path('data/pic/src/640/base/base_incorrect_green_52.JPG'),
 Path('data/pic/src/640/base/base_correct_stone_12.JPG'),
 Path('data/pic/src/640/base/base_incorrect_wood2_22.JPG'),
 Path('data/pic/src/640/base/base_incorrect_blue_47.JPG'),
 Path('data/pic/src/640/base/base_incorrect_white_60.JPG'),
 Path('data/pic/src/640/base/base_incorrect_wood_10.JPG'),
 Path('data/pic/src/640/base/base_correct_wood2_32.JPG'),
 Path('data/pic/src/640/base/base_incorrect_stone_1.JPG'),
 Path('data/pic/src/640/base/base_correct_wood2_8.JPG

In [5]:
def get_label(fn):
    return fn.parent.name

In [6]:
fnames = get_image_files(path)
fn = fnames[0]

fn, get_label(fn)

(Path('data/pic/src/640/base/train/incorrect/base_incorrect_green2_21.JPG'),
 'incorrect')

In [21]:
def get_loaders(size, trans_mult = 1., bs = 64):
    dBlock = DataBlock(blocks = (ImageBlock, CategoryBlock),
                   get_items = get_image_files,
                   get_y = get_label,
                   splitter = GrandparentSplitter(),
                   item_tfms = Resize((size, int(1.6 * size)), method = 'squish'),
                   batch_tfms = aug_transforms(#size = (size,int(size * 1.6)),
                                                   max_rotate = 10.,
                                                   min_zoom = 1.,
                                                   max_zoom = 1.,
                                                   p_affine = 1.,
                                                   p_lighting = 0.8,
                                                   max_lighting = 0.3,
                                                   max_warp = 0.2,
                                                   mult = trans_mult
                                                   )
                  )
    
    return dBlock.dataloaders(path, bs = bs, shuffle_train = True)

In [22]:
dls = get_loaders(420, bs = 16)
#dls.show_batch()

In [9]:
#dls.valid.show_batch(max_n = 3)

In [24]:
learn = cnn_learner(dls, resnet34, metrics=[accuracy, error_rate]).to_fp16()

In [25]:
learn.fine_tune(20)

epoch,train_loss,valid_loss,accuracy,error_rate,time
0,1.266365,1.539485,0.519481,0.480519,00:11


KeyboardInterrupt: 

In [17]:
learn.get_preds(dl=dls.valid, with_decoded = True)[1:]

(tensor([0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0,
         1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0,
         1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0,
         1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1,
         0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1,
         1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0,
         0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1,
         0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0,
         0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1,
         1]),
 tensor([0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0,
         1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0,
         1, 0,

In [ ]:
x,y = dls.valid.one_batch()
b = dls.valid.one_batch()
dls.decode(b)

In [ ]:
lr_min, _ = learn.lr_find()

In [ ]:
learn.save('res18-224-pre-val') # 0.460, 0.396, 0.8548 / 0.1451

In [ ]:
learn.load('res18-224-pre-val')
learn.fine_tune(10, base_lr = 2e-3, wd = 0.1, freeze_epochs = 5)

In [ ]:
learn.save('res18-224-cold-val')

In [ ]:
learn.unfreeze()
learn.lr_find()

In [ ]:
new_lr = slice(7e-7,1e-2)

learn.fit_one_cycle(10, new_lr, wd = 0.1)

In [ ]:
learn.save('res18-224-hot-val')

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)

losses,idxs = interp.top_losses()

In [ ]:
plt.rcParams['figure.dpi'] = 300
interp.plot_top_losses(9, figsize=(15,11))

In [ ]:
interp.plot_confusion_matrix(figsize=(12,12), dpi=60)

### Transfer learning
There is no obvious group of pictures the learner performs poorly on, so increase imagesize.

In [ ]:
dls = get_loaders(420, bs = 32) # half batchsize for image size increase

learn = cnn_learner(dls, 
                    resnet18, 
                    metrics=[accuracy, error_rate])

In [ ]:
learn.load('res18-224-hot-val');

In [ ]:
learn.lr_find()

In [ ]:
learn.fine_tune(15, 1e-3)

In [ ]:
learn.save('res18-420-cold-val')

In [ ]:
learn.lr_find()

In [ ]:
learn.fit_one_cycle(15, slice(3e-6,1e-3))

In [ ]:
learn.save('res18-420-hot-val')

In [ ]:
dls = get_loaders(640, bs = 16)

learn.dls = dls

In [ ]:
learn.freeze()
learn.lr_find()

In [ ]:
learn.fine_tune(15, 1e-3)

In [ ]:
learn.save('res18-640-cold-val')

In [ ]:
learn.unfreeze()
learn.lr_find()

In [ ]:
learn.fit_one_cycle(20, slice(1e-6,1e-3))

In [ ]:
learn.save('res18-640-hot-val')